IEKF implementation for differential drive vehicle.

Authors: Håvard Brenne and Marcus Lerfald

In [ ]:
from math import sin, cos, remainder
import numpy as np
    
class Vehicle:
    def __init__(self, measurement_stds, T=0.01):
        # x: x, y, theta, v
        # u: a, omega
        self.T = T
        self.measurement_stds = measurement_stds

    def f(self, x, u, enable_noise=True):
        # Zero-order hold discretization. x_dot ≈ 1/dt(x_k+1 - x_k)
        # Ad = I + A*dt, Bd = B*dt
        # x_dot = v*cos(theta)
        # y_dot = v*sin(theta)
        # theta_dot = omega
        # v_dot = a
        B = np.array([
            [0, 0],
            [0, 0],
            [0, 1],
            [1, 0]
        ])
        A = np.zeros((x.size, x.size))
        A[0,3] = cos(x[2])
        A[1,3] = sin(x[2])
        
        Ad = np.eye(x.size) + A*self.T
        Bd = B*self.T
        x_new = Ad @ x + Bd @ (u + enable_noise * np.random.normal(0, np.flip(self.measurement_stds[-2:])))
        x_new[2] = remainder(x_new[2], 2*np.pi)  # Limit to -pi~pi
        return x_new

    def F(self, x):
        return np.array([
            [1, 0, -x[3]*self.T*sin(x[2]), self.T*cos(x[2])],
            [0, 1, x[3]*self.T*cos(x[2]), self.T*sin(x[2])],
            [0, 0, 1, 0],
            [0, 0, 0, 1]
        ])

    def g(self, x, enable_noise=True):
        Cd = np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])
        return Cd @ x + enable_noise*np.random.normal(0, self.measurement_stds[:2]).T

    def G(self):
        return np.array([
            [1, 0, 0, 0],
            [0, 1, 0, 0]
        ])

To compare the IEKF and the EKF, we consider the case of normal and high noise levels.

In [ ]:
# Low noise in position, angular rate and linear acceleration
# Long simulation

measurement_stds = np.array([0.3,0.3,0.05,0.1])
num_iterations = 50000

In [ ]:
# High position, angular rate and linear acceleration noise
# Long simulation

measurement_stds = np.array(([3.0,3.0,0.5,1.0]))
num_iterations = 50000

In [ ]:
from math import remainder
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(0)   # For repeatability

num_states = 4
num_measurements = 2
max_iekf_iterations = 10

position_update_freq = 100
timesteps = np.arange(0,num_iterations)

trajectory = np.empty((0,num_states), float)
ground_truth = np.empty((0,num_states), float)
NIS = np.empty((0,1), float)
NEES = np.empty((0,1), float)

# Initialize model
x0 = np.array([0]*num_states)
P0 = 1e-4*np.eye(num_states)
gt = x0

vehicle = Vehicle(measurement_stds=measurement_stds)

u = np.vstack((num_iterations*[0.1], num_iterations*[0.5]))
u[0, num_iterations//4:3*num_iterations//4] = 0 # Drive in a circle for 3/5 the simulation

x_prev = x0
P_prev = P0

Q = np.diag([1e-1,1e-1,1e-2,5e-2])**2
R = np.diag(measurement_stds[:2])**2

G = vehicle.G()

k = 0
while k <= num_iterations - 1:
    
    # Prediction step
    P_pred = vehicle.F(x_prev) @ P_prev @ vehicle.F(x_prev).T + Q
    x_pred = vehicle.f(x_prev, u[:,k])
    
    # Kalman gain and covariance
    S = G @ P_pred @ G.T + R    # Innovation covariance
    K = np.linalg.solve(S.T, (P_pred @ G.T).T).T   # Faster than P_pred @ G.T @ np.linalg.inv(G @ P_pred @ G.T + R)
    P = (np.eye(num_states) - K @ G) @ P_pred
    P_prev = P
    
    x_noisy = vehicle.f(gt, u[:,k], enable_noise=True)
    gt = vehicle.f(gt, u[:,k], enable_noise=False)
    if (k % position_update_freq == 0):
        # Update step
        x_op = x_pred
        y = vehicle.g(x_noisy, enable_noise=True)
        i = 1
        while i <= max_iekf_iterations:
            if i != 1:
                x_op = x
            y_op = vehicle.g(x_op, enable_noise=False)
            x = x_pred + K @ (y - y_op - G @ (x_pred - x_op))
            x[2] = remainder(x[2], 2*np.pi)
            if np.linalg.norm(x - x_op) < 1e-10:
                break
            i += 1
        y = vehicle.g(x, enable_noise=True)
        g = vehicle.g(x, enable_noise=False)
        innovation = y - g
        x_prev = x

        NIS = np.vstack((NIS, innovation.T @ np.linalg.solve(S, innovation)))
    else:
        x = x_pred
        x[2] = remainder(x[2], 2*np.pi)
        x_prev = x

    # Store estimated trajectory
    trajectory = np.vstack((trajectory, x))

    # Store ground truth
    ground_truth = np.vstack((ground_truth, gt))
    estimation_error = gt - x
    estimation_error[2] = remainder(estimation_error[2], 2*np.pi) # Limit to -pi~pi

    NEES = np.vstack((NEES, estimation_error.T @ np.linalg.solve(P, estimation_error)))

    # Count iterations
    k+=1
ANIS = np.mean(NIS)
ANEES = np.mean(NEES)

In [ ]:
from scipy.stats.distributions import chi2

# Using https://arxiv.org/abs/1807.08855 for tuning
print(f"ANIS: {ANIS:.2f}")
print(f"ANEES: {ANEES:.2f}")

confidence = 0.95
alpha = 1 - confidence
NEES_bounds = [chi2.ppf(alpha/2, df=num_states), chi2.ppf(1-alpha/2, df=num_states)]
NIS_bounds = [chi2.ppf(alpha/2, df=num_measurements), chi2.ppf(1-alpha/2, df=num_measurements)]

NIS_within_bounds = ((NIS_bounds[0] < NIS) & (NIS < NIS_bounds[1])).sum()/NIS.size
NEES_within_bounds = ((NEES_bounds[0] < NEES) & (NEES < NEES_bounds[1])).sum()/NEES.size

print(f"NIS within {confidence*100:.2f} percentile: {NIS_within_bounds*100:.2f}%")
print(f"NEES within {confidence*100:.2f} percentile: {NEES_within_bounds*100:.2f}%")

print(f"RMSE in position estimate: {np.sqrt(np.mean((trajectory[:,:2]-ground_truth[:,:2])**2)):.3f}")

print(f"Maximum error in position estimate: {np.max(np.sqrt((trajectory[:,:2]-ground_truth[:,:2])**2)):.2f}")

# Plotting
plt.rcParams['figure.figsize'] = [15, 15]
fig, axs = plt.subplots(4, 2)
axs[0,0].plot(trajectory[:,0], trajectory[:,1])
axs[0,0].plot(trajectory[::position_update_freq,0], trajectory[::position_update_freq,1], 'bo')
axs[0,0].set_title('Estimated position in XY plane')
axs[0,1].plot(trajectory[:,0], trajectory[:,1], ground_truth[:,0], ground_truth[:,1])
axs[0,1].plot(trajectory[::position_update_freq,0], trajectory[::position_update_freq,1], 'bo')
axs[0,1].plot(ground_truth[::position_update_freq,0], ground_truth[::position_update_freq,1], 'ro')
axs[0,1].set_title('Estimated position in XY plane vs ground truth')
states = ["x position","y position",r"$\theta$","velocity"]
for i in range(num_states):
    axs[1+i//2, i%2].plot(timesteps, trajectory[:,i], ground_truth[:,i])
    axs[1+i//2, i%2].set_title(f'Estimated {states[i]} vs ground truth')
axs[3,0].plot(timesteps[::position_update_freq], NIS[:,0])
axs[3,0].axhline(y = num_measurements, color = 'r', linestyle = '-')
axs[3,0].set_title('NIS')
axs[3,0].axhline(y = num_measurements, color = 'r', linestyle = '-')
[axs[3,0].axhline(y = i, color = 'b', linestyle = '-') for i in NIS_bounds]
axs[3,1].plot(timesteps, NEES[:,0])
axs[3,1].set_title('NEES')
axs[3,1].axhline(y = num_states, color = 'r', linestyle = '-')
[axs[3,1].axhline(y = i, color = 'b', linestyle = '-') for i in NEES_bounds]
pass

For the IEKF, we see results similar to the EKF. When increasing the noise in position measurements, angular velocity and linear acceleration, the estimates will be degraded and deviate sooner.
 
For the simulation with low noise, the IEKF and EKF have almost identical RMSE. The NIS and NEES are also very similar.
The reason for this is the linear measurement model. Since $G=C_d$ is constant, we do not get a new Kalman gain each iteration and our IEKF correction steps reduces to a regular EKF correction step:

Let $x^k$ indicate correction iteration $k$ of the IEKF.
$$\hat x^2 = \check x + K(y-y_{op}^2 - G(\check x - x_{op}^2) \ | \ y_{op} = g(x_{op}, 0)=G x_{op}$$
$$=\check x + K(y-G x_{op}^2 - G(\check x - x_{op}^2) \ | \ x_{op}^k = \hat x^{k-1}$$
$$=\check x + K(y-G \hat x^1 - G(\check x - \hat x^1)$$
$$=\check x + K(y- G \check x + G \hat x^1 - G \hat x^1)$$
$$=\check x + K(y- G \check x)$$
$$=\check x + K(y- C_d \check x)$$

Which is the EKF correction step.
At this point, the IEKF will stop iterating, because there is no change in $\hat x$:
$$\hat x^2 - \hat x^1 = \check x + K(y- G \check x) - (\check x + K(y-y_{op}^1 - G(\check x - x_{op}^1))$$
$$= \check x + K(y- G \check x) - \check x - K (y-g(\check x, 0)) + G(\check x - \check x)$$
$$=0$$

For models with stronger nonlinearities, this is not the case and we would expect a significant performance increase with an IEKF.
This is the main advantage of the IEKF, as the linearization point can converge to the mode of the full Bayesian posterior, instead of the mean as with the EKF, resulting in lower divergance rate.




